Shamir Secret Sharing:

- Enter the secret, s, and the minimum number of recipients, t, of n shares.

- Let p > max(s, n), with $\mathbb{F}_p$ as our finite field. 

- Generate random coefficients for a polynomial of degree t-1, f(x), with the y-intercept as the secret text.

- Compute n points on the polynomial, (x, f(x)), with $x \neq 0$.

- Give each participant a unique point.

- To reconstruct the secret, use Lagrange interpolation formula with t many points of interpolation, and evaluate it at f(0) mod p.

In [3]:
import pandas as pd 
import sagemath as sg 
import random as rd
import numpy as np
import plotly.express as px
from random import randint
from math import gcd
import gmpy2
from gmpy2 import mpz, is_prime, gcdext, invert
import sympy as sp
from sympy import isprime, factorint, mod_inverse
import numpy.polynomial.polynomial as poly

Encoding

In [ ]:
secret = 67 # item we are encrypting 
min_reps = 5 # number of reciepeients
n = 100 # number of sample size aka crowd 

p = max(secret, n) + 1  # lower bound of said prime number
def getPrime(p): # sets p = to a prime number
    k = False  # initial condition of k

    while k == False: # '''starts loop, runs "Boulian" condition until prime number, when it finds 
        # prime number, breaks loop, when not adds 1 and continues'''
        if isprime(p) == True: 
            k = True
        else:
            p = p +1   
        
    return p

prime = getPrime(p) # runs our previously established function
print(f'We are using Z mod {prime} as our finite field.') # prints message with our prime number


def polynomialMaker(t,prime): 
    polylist = [secret] # initializes your array and sets our first value to our secrets
    for i in range(t-1): # number of reciepents (t / min_reps) -1
        polylist = polylist + [randint(0,prime)] #''' appends (ends our loop after our i = min_reps/t a value 
        # between 0 and your prime number to the end of your polylist'''
    eq = poly.Polynomial(polylist) # makes the polynomial output readable
    return eq # ends our loop 

# Printing info presented in previous loop
encodingPoly = polynomialMaker(min_reps,prime)
print('\nPolynomial:')
display(encodingPoly)  


# This is the part where we are building out the L(i)
def splitter(eq, n, prime):  
    points = pd.DataFrame(columns=['X', 'f(X)']) # establishes our coloumn names
    list = [] # starts our empty list
    for i in range(n): # establishes our range with n = # of people in crowd
        k = randint(1,n) # gives random integer between 1 and 100 = n
        if k in list: 
            continue # so if k in our list we continure our loop
        else:
            list = list + [k] # if not in our list we add k to our lsit
            points.loc[i, 'X' ] = k # i is our recpients, lines our random int. X up with proper reciepent
            num = eq(k) # polynomial in terms of k
            points.loc[i, 'f(X)'] = num # i is our recpients, lines our F(X) number up with proper reciepent
    
    return points # ends loop

ciphertext = splitter(encodingPoly, n, prime) # takes previous loop and inputs it into a nice table
display(ciphertext) # prints out our information

We are using Z mod 101 as our finite field.

Polynomial:


Polynomial([67., 91., 25., 12., 74.], domain=[-1,  1], window=[-1,  1], symbol='x')

,X,f(X)
0,99,7120308163.0
1,5,48897.0
2,15,3793807.0
3,9,497173.0
4,77,2606958729.0
...,...,...
87,18,7848013.0
88,74,2224013013.0
89,28,45770183.0
97,16,4906739.0


Decoding:

In [14]:
def modinv(a, p):
    return pow(a, p - 2, p)

def lagrange_symbolic(df, p):
    x = sp.Symbol('x')
    xs = [int(v) for v in df['X']]
    ys = [int(v) for v in df['f(X)']]
    n = len(xs)

    poly = 0
    for i in range(n):
        xi, yi = xs[i], ys[i]
        term = yi
        numerator = 1
        denominator = 1
        Li = 1
        for j in range(n):
            if i != j:
                xj = xs[j]
                numerator *= (x - xj)
                denominator *= (xi - xj) % p
        Li = numerator * modinv(denominator, p)
        poly += term * Li

    # Expand and reduce modulo p
    poly = sp.expand(poly)
    coeffs = sp.Poly(poly, x, modulus=p).all_coeffs()  # highest degree first

    # Convert to ascending order
    return [int(c) for c in reversed(coeffs)]

shift_factor = 0

decodingPoly = lagrange_symbolic(ciphertext.head(min_reps+shift_factor), prime)
eq = poly.Polynomial(decodingPoly)
print(f'Lagrange Polynomial is {eq}')
print(f'\nSafetext was {(eq(0)%prime)}')


Lagrange Polynomial is -34.0 + 17.0 x + 0.0 x**2 + 3.0 x**3 - 43.0 x**4

Safetext was 67.0
